In [1]:
import dynamiqs as dq
import scipy as sp
import jax
import jax.numpy as jnp # the JAX version of numpy
from matplotlib import pyplot as plt
import scipy.constants
jax.devices()
dq.set_device('cpu')

dq.set_precision('simple')

In [4]:
na = 20
nb = 5
ω_a0 = 2*jnp.pi*5.26
ω_b0 = 2*jnp.pi*7.70
 
φ_a = 0.06
φ_b = 0.29

h = 6

Kb = 10
E_J = 42.76e3* h
 
ΔE_J = 0.47e3 * h
T = 4

t_save = jnp.linspace(0, T, 50)  # save times

ω_d = 2*jnp.pi*7.623 #7.623
ω_p = 2*jnp.pi*2.891 #2.891
 
ϵ_d = -2*jnp.pi*3.815 #This is MHz but rest is GHz , 3.815 GHz
ϵ_p = 0.122
a,b = dq.destroy(na,nb)

phi_hat = φ_a * (a + dq.dag(a)) + φ_b * (b + dq.dag(b))
H_0 = ω_a0* dq.dag(a) @ a + ω_b0* dq.dag(b) @ b 

H_ATS = dq.modulated(lambda t: -2* E_J*(jnp.sin(ϵ_p * jnp.cos(ω_d * t))), dq.sinm(phi_hat)) + dq.modulated(lambda t : 2 * ΔE_J * jnp.cos(ϵ_p * jnp.cos(ω_d * t)), dq.cosm(phi_hat))

H_d = dq.modulated(lambda t: 2 * ϵ_d * jnp.cos(ω_d * t), (b + dq.dag(b)))

H = H_0 + H_ATS + H_d 

C:\Users\gebha\AppData\Roaming\Python\Python313\site-packages\equinox\_module.py:1096: UserWarning: A `SparseDIAQArray` has been converted to a `DenseQArray` while computing its matrix exponential.
  return self.__func__(self.__self__, *args, **kwargs)


In [ ]:
psi0 = dq.fock((na,nb), (0,0))  # coherent state

res = dq.mesolve(H, [jnp.sqrt(Kb)*b], psi0, t_save)

trace_a = dq.ptrace(res.states, 0,res.states.dims) # trace of a

dq.plot.wigner_gif(trace_a, fps=120)

|▎         |   3.6% ◆ elapsed 44.88s ◆ remaining 32m52sERROR:2025-02-01 21:55:07,436:jax._src.callback:97: jax.pure_callback failed
Traceback (most recent call last):
  File "C:\Users\gebha\AppData\Roaming\Python\Python313\site-packages\ipykernel\iostream.py", line 142, in _event_pipe
    event_pipe = self._local.event_pipe
                 ^^^^^^^^^^^^^^^^^^^^^^
AttributeError: '_thread._local' object has no attribute 'event_pipe'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\gebha\AppData\Roaming\Python\Python313\site-packages\jax\_src\callback.py", line 95, in pure_callback_impl
    return tree_util.tree_map(np.asarray, callback(*args))
                                          ~~~~~~~~^^^^^^^
  File "C:\Users\gebha\AppData\Roaming\Python\Python313\site-packages\jax\_src\callback.py", line 72, in __call__
    return tree_util.tree_leaves(self.callback_func(*args, **kwargs))
                                 ~

In [4]:
omega_a = 1.0 # eigenfrequency of the mode
epsilon_d = 2.0 # drive strength
omega_d = 1.0 # resonant drive
kappa_a = 1.2 # single-photon loss rate

def alpha(t):
    num = -1j*ϵ_d*jnp.exp(1j*ω_d*t)
    denom = kappa_a/2 + 1j*(omega_a - omega_d)
    return num/denom


def dalpha(t):
    return 1j*omega_d*alpha(t)




In [9]:
I = dq.eye(na, nb)

lop = [dq.timecallable(lambda t: jnp.sqrt(kappa_a)*(a + alpha(t)*I)), jnp.sqrt(Kb)*b]

psi0 = dq.todm(dq.coherent(na, 0.0) & dq.coherent(nb, 0.0))

t_save = jnp.linspace(0, T, 50)  # save times

print(psi0)


QArray: shape=(100, 100), dims=(20, 5), dtype=complex64, layout=dense
[[1.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
 ...
 [0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]]


In [ ]:
res_displaced = dq.mesolve(H, [lop], psi0, t_save)


In [ ]:
dq.plot.wigner_gif(res_displaced.states, fps = 12)
